# Part 2:  Segmentation

# Setup

### Our usual imports and initializing napari

In [43]:
import numpy as np
import pandas as pd
import napari
import tifffile
import skimage as ski
import scipy.ndimage as ndi
import glob
import plotly.express as px
import cellpose.models as models
import matplotlib.pyplot as plt
import cv2
import dask

In [44]:
viewer = napari.Viewer()

### Support functions for this notebook

Skimage's rolling ball background subtraction (ski.restoration.rolling_ball), is slow and does not work as well as ImageJ's.  This function more closely matches ImageJ's rolling ball background subtraction.

In [74]:
def backsub_2D(inp, radius=60):
    """
    This function performs background subtraction on a 2D image using a morphological operation called "top-hat".

    Parameters:
    inp (numpy.ndarray): The input 2D image.
    radius (int, optional): The radius of the structuring element used for the top-hat operation. Default is 60.

    Returns:
    rtn (numpy.ndarray): The result image after background subtraction.

    How it works:
    1. It first creates an elliptical structuring element with the specified radius.
    2. It then applies a Gaussian blur to the input image to reduce noise.
    3. It performs a top-hat operation on the blurred image. The top-hat operation is the difference between the input image and its opening. It is used to isolate small elements and details from the larger ones.
    4. It subtracts the result of the top-hat operation from the input image to get the final result.
    5. Finally, it clips the result to have a minimum value of 0 (to remove any negative values that might have resulted from the subtraction).
    """
    filterSize =(radius, radius)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, filterSize)
    blurred = cv2.GaussianBlur(inp, (5, 5), 0)
    tophat_img = cv2.morphologyEx(blurred, cv2.MORPH_TOPHAT, kernel)
    rtn = inp.astype(np.single) - (blurred-tophat_img)
    rtn = np.clip(rtn, 0, np.inf)
    return rtn

In [75]:
def remove_objects(label_image, area_min, area_max):
    """
    This function removes objects from a labeled image based on their area.

    Parameters:
    label_image (numpy.ndarray): A 2D array where each unique non-zero value represents a unique object/region.
    area_min (int): The minimum area threshold. Objects with area less than this will be removed.
    area_max (int): The maximum area threshold. Objects with area more than this will be removed.

    Returns:
    cleaned_label_image (numpy.ndarray): A 2D array similar to label_image but with small and large objects removed.

    How it works:
    1. It first calculates the properties of each labeled region in the input image using skimage.measure.regionprops.
    2. It then creates a boolean mask of the same size as the input image. This mask is True for pixels belonging to regions that are too small or too large.
    3. Finally, it applies this mask to the input image, setting the labels of the small and large regions to zero, effectively removing them from the image.
    """
    # Get properties of labeled regions
    props = ski.measure.regionprops(label_image)

    # Create a boolean mask where True indicates a region is too small or too large
    object_mask = np.zeros_like(label_image, dtype=bool)
    for prop in props:
        if prop.area < area_min or prop.area > area_max:
            object_mask[label_image == prop.label] = True

    # Apply the mask to the label image
    cleaned_label_image = np.where(object_mask, 0, label_image)

    return cleaned_label_image

In [114]:
def shrink_labels(label_image, shrinkage=3):
    """
    This function shrinks the regions in a labeled image using morphological erosion.

    Parameters:
    label_image (numpy.ndarray): A 2D array where each unique non-zero value represents a unique object/region.
    shrinkage (int, optional): The size of the structuring element used for the erosion operation. Default is 3.  This default shrinks by 1 pixel in each direction.

    Returns:
    shrunken_label_image (numpy.ndarray): A 2D array similar to label_image but with all regions shrunken.

    How it works:
    1. It first creates a circular structuring element of the specified size.
    2. It then initializes an empty image of the same size as the input image.
    3. It calculates the properties of each labeled region in the input image using skimage.measure.regionprops.
    4. For each region, it extracts the region from the original label image, erodes it using the structuring element, and then updates the corresponding region in the shrunken_label_image.
    5. The erosion operation shrinks the region by removing pixels from its boundary.
    """
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (shrinkage, shrinkage))

    shrunken_label_image = np.zeros_like(label_image)
    regions = ski.measure.regionprops(label_image)

    for region in regions:
        lbl = region.label
        minr, minc, maxr, maxc = region.bbox

        minr = max(minr - 1, 0)
        minc = max(minc - 1, 0)
        maxr = min(maxr + 1, label_image.shape[0])
        maxc = min(maxc + 1, label_image.shape[1])

        # Extract the region from the original label image
        region_mask = label_image[minr:maxr, minc:maxc] == lbl

        # Erode the region
        eroded_region_mask = cv2.erode(region_mask.astype(np.uint8), kernel)

        # Update the shrunken_label_image
        shrunken_label_image[minr:maxr, minc:maxc][eroded_region_mask == 1] = lbl

    return shrunken_label_image

These support functions, since they are defined in THIS file and not in an imported module, can be called simply as backsub_2D(), they do not have a "ski." prefix

### Loading the image for this notebook

In [48]:
img = tifffile.imread('files/C-hela-cells.tif')
img.shape

(512, 672, 3)

And visualize in napari with the appropriate names and colors

In [49]:
viewer.layers.clear()
viewer.add_image(img, name=['lysosomes', 'mitocondria', 'nucleii'], colormap=['red', 'green', 'blue'], channel_axis=2)

[<Image layer 'lysosomes' at 0x2373d5a3b80>,
 <Image layer 'mitocondria' at 0x237895a06d0>,
 <Image layer 'nucleii' at 0x23789605390>]

# Pre-processing

### Standard pipeline:  subtract background, gaussian blur, threshold

To make things easier on ourselves, we split the 3 channels into 3 separate variables.  The contour of the 3 channels is very different:  the lysosomes are small puncta, the mitos are large networks with holes in them, and the nucleii are very large blobs.  We would not want to use the same rolling ball radius for all 3 channels.

In [50]:
lyso = img[:,:,0]
mitos = img[:,:,1]
nucleii = img[:,:,2]

The nucleii are much larger than the lysos or mitos, so we will use a much larger rolling ball radius.

In [51]:
lyso_backsub = backsub_2D(lyso, radius=20)
mito_backsub = backsub_2D(mitos, radius=20)
nucleii_backsub = backsub_2D(nucleii, radius=200)

viewer.add_image(lyso_backsub, name='lysosomes_backsubbed', colormap='red', blending='additive')
viewer.add_image(mito_backsub, name='mito_backsubbed', colormap='green', blending='additive')
viewer.add_image(nucleii_backsub, name='nucleii_backsubbed', colormap='blue', blending='additive')


<Image layer 'nucleii_backsubbed' at 0x2372dd4c190>

For segmentation we will use the nucleii primarily, we'll apply some blurring to make sure we avoid holes and small puncta.

In [52]:
blurred = ndi.gaussian_filter(nucleii_backsub, 10)
viewer.add_image(blurred, name='blurred', colormap='gray', blending='additive')

<Image layer 'blurred' at 0x2379ae25ff0>

# Simple Segmentation

## Thresholding

Mousing over the image (make sure you have the "blurred" layer selected) we can see that nucleii have pixel intensities > 300, so we will use that as our threshold and visualize the binary image.

In [53]:
thresholded = blurred > 300
viewer.add_image(thresholded, name='thresholded', colormap='gray', blending='additive')

<Image layer 'thresholded' at 0x2379ae58a60>

With the "thresholded" layer selected, try mousing over the pixels.  Turns out python, whenever given an expression of A > B, returns a boolean array of all "True" or "False".  Napari is smart enough to turn these into 1 and 0.

## Label images

scipy.ndimage has a function called label that will take a binary image and return a "labeled" image.  A label image is an image where each pixel is assigned a number, and all pixels with the same number are connected.  This is exactly what we want for segmentation.  The function actually returns two arguments, the label_img and the number of objects it found.

In [54]:
label_img, number_objects = ndi.label(thresholded)

If we add the label_img to napari, we can see each individual object is a different intensity

In [55]:
viewer.add_image(label_img, name='label_img', colormap='gray', blending='additive')

<Image layer 'label_img' at 0x2379b2f0130>

...but this is not very conducive to seeing separation between objects if they have a label value that is very similar.  Napari has a nice feature where you can instead .add_labels(label_img) and it will automatically assign a random color to each label.

In [56]:
viewer.layers.remove('label_img')
viewer.add_labels(label_img, name='label_img')

<Labels layer 'label_img' at 0x23788ebac80>

## Regionprops (analyze particles)

### Regionprops (quantifying labels)

We have a binary image (thresholded) that ImageJ would normally use for Analyze Particles, but we have managed to improve on it with a label image (label_img).  Label images are superior, as if two objects are touching in a binary image, ImageJ lumps them together into a single object.  With labeled images (as we shall see with cellpose), you can have objects touching but still be separated (each gets a different intensity value assigned to it).  Now we want to quantify each object, and we can do that with skimage.measure.regionprops_table.  We have to specify what properties we want to collect, it can be computationally expensive to collect all of them, so we will just collect the ones we need.

The available properties are:  https://scikit-image.org/docs/dev/api/skimage.measure.html#skimage.measure.regionprops

The most useful ones are:  

label (the index of the object in the image), 

area (in number of pixels), 

centroid (the z/y/x position of the center of the object), 

mean_intensity, max_intensity, min_intensity, 

perimeter,  

eccentricity (how extended the object is, ranging from 0->1, with a circle having ecc=0), 

orientation (the angle the object makes in radians), 

axis_major_length, axis_minor_length

Unfortunately regionprops_table returns a dictionary instead of a simple table, so we have to do some extra work to get it into a table.  We'll use pandas.DataFrame.from_dict to convert the dictionary into a table.

In [57]:
results = pd.DataFrame.from_dict(ski.measure.regionprops_table(label_img, properties=['label', 'area', 'centroid', 'orientation', 'eccentricity']))
results

,label,area,centroid-0,centroid-1,orientation,eccentricity
0,1,14404.0,179.556651,463.463968,0.109041,0.787659
1,2,15433.0,171.781572,298.301173,-1.419267,0.549767
2,3,14937.0,279.708040,140.079199,-0.609245,0.726105
3,4,14258.0,408.910296,333.062141,-0.593316,0.736625


'centroid-0' is the y position, 'centroid-1' is the x position.  Orientation in radians is not intuitive, so let's fix that.

In [58]:
results['orientation'] = (90+results['orientation']/np.pi*180)
results

,label,area,centroid-0,centroid-1,orientation,eccentricity
0,1,14404.0,179.556651,463.463968,96.247604,0.787659
1,2,15433.0,171.781572,298.301173,8.681978,0.549767
2,3,14937.0,279.708040,140.079199,55.092811,0.726105
3,4,14258.0,408.910296,333.062141,56.005484,0.736625


Comparing to the image, we should see that label 2 (centered at 171/298) is pretty flat (8 degrees), and label 1 (centered at 180/463) is mostly vertical (96 degrees).

### Regionprops (quantifying intensities)

Note that the only argument we gave to ski.measure.regionprops_table that had an image in it was label_img which does not include any information about the raw intensities of our original image:  just shape information.  

What if we want to quantify the intensities of the lysosome or mitocondrial channel?  ski.measure.regionprops_table lets use give an intensity image as an argument.

In [59]:
mito_results = pd.DataFrame.from_dict(ski.measure.regionprops_table(label_img, mito_backsub, properties=['label', 'area', 'centroid', 'mean_intensity']))
mito_results

,label,area,centroid-0,centroid-1,mean_intensity
0,1,14404.0,179.556651,463.463968,65.834702
1,2,15433.0,171.781572,298.301173,66.851036
2,3,14937.0,279.708040,140.079199,73.726715
3,4,14258.0,408.910296,333.062141,82.320663


Compare to the original image and mito_backsubbed, does this make sense?  Note a fun feature of napari label layers:  you can change the "contour" argument to 1, and it will show just the outline of the labels.

Regionprops is a very powerful tool, we can write our own custom functions to perform some kind of analysis on each object.  For instance you could write something to take find the 90th percentile of intensity in each object and the 10th percentile, useful for looking at things like how punctate the signal is in an object.

# Watershed segmentation

## Masking cells vs background

This is fine for finding the nucleii, but what if we wanted to find the whole cell, and keep them separate?  First we need to find a binary image of the cells.  The raw, unbackground subtracted lyso channel seems like a reasonable place to start.  First we'll blur it a little bit.

In [17]:
smoothed_lyso = ndi.gaussian_filter(lyso, 10)
viewer.add_image(smoothed_lyso, name='smoothed_lyso', colormap='gray', blending='additive')

<Image layer 'smoothed_lyso' at 0x2372e571e70>

If we adjust the contrast lower limit to around ~350 we can see the cells, so we'll use that as our threshold.

In [18]:
masked_lyso = smoothed_lyso>350
viewer.add_image(masked_lyso, name='masked_lyso', colormap='gray', blending='additive')

<Image layer 'masked_lyso' at 0x2372df7de40>

## Separating the cells

Now we have a binary image of where there is cell vs no cell, but we have not separated them into individual cells.  To do so we are going to combine our nuclear mask with a little trick:  we are going to use the distance transform of the cell mask.  The distance transform is a measure of how far each pixel is from the nearest "edge" of the cell mask.  We can use this to "push" the cells apart from each other.

The distance transform is actually a pretty simple concept:  for each pixel measure how many pixels away a background (False) pixel is and that is your new intensity.

In [19]:
edt = ndi.distance_transform_edt(masked_lyso)
viewer.add_image(edt, name='edt', colormap='gray', blending='additive')

<Image layer 'edt' at 0x2372eba4460>

ski.segmenation.watershed takes an image that can be used to decide when to break two objects apart (the edt), our seed image with the nucleii labeled (label_img), and a final mask that limits how far we can grow our cells (masked_lyso).

In [20]:
watershedded = ski.segmentation.watershed(-edt, label_img, mask=masked_lyso)
viewer.add_labels(watershedded, name='watershedded', blending='additive')

<Labels layer 'watershedded' at 0x2372f228370>

If we turn the watershedded onto contour=1, and then adjust the contrast on the EDT, we can see kind of what it is doing.  It breaks the boundary between two cells where the distance transform is the smallest.

## Quantifying the cells

Watershedded is a label image, but that contains the whole cells now.  We can use the same regionprops_table function to quantify the cells.

In [21]:
results = pd.DataFrame(ski.measure.regionprops_table(watershedded, lyso_backsub, properties=('label', 'area', 'centroid', 'mean_intensity')))
results

,label,area,centroid-0,centroid-1,mean_intensity
0,1,44235.0,188.152933,499.062462,71.550331
1,2,50295.0,131.969520,319.039407,54.592186
2,3,42868.0,243.536437,136.338947,73.122444
3,4,38548.0,397.226497,366.374883,94.249893


Let's say we want to also quantify the mitos, we will create a new table, but take the mito_intensity and just add it to the results table we already have by defining a new column 'mito_mean_intensity'

In [22]:
lyso_results = pd.DataFrame(ski.measure.regionprops_table(watershedded, mito_backsub, properties=('label', 'area', 'centroid', 'mean_intensity')))
results['mito_mean_intensity'] = lyso_results['mean_intensity']
results

,label,area,centroid-0,centroid-1,mean_intensity,mito_mean_intensity
0,1,44235.0,188.152933,499.062462,71.550331,101.695312
1,2,50295.0,131.969520,319.039407,54.592186,108.867981
2,3,42868.0,243.536437,136.338947,73.122444,99.088104
3,4,38548.0,397.226497,366.374883,94.249893,106.858253


# Visualizing results

### Plotting

Obviously we can plot results using plotly.  For those that don't know:  plotly functions take a datatable, and then you specify which columns you want to plot on which axis.  You can also specify which column you want to use to color the points, and which column you want to use to size the points.

In [23]:
px.bar(results, x='label', y='area', width=400)

In [24]:
px.bar(results, x='label', y='mean_intensity', title='Lyosome background intensity', width=400)

### With images

But this is boring!  We can visualize this in a much more exciting way using napari.

The ski.util.map_array() function lets us take a label image (in this case watershedded), and map intensities onto each object.  We can use this to visualize the lysosome intensity of each cell.

In [25]:
intensity_img = ski.util.map_array(watershedded, results['label'].values, results['mean_intensity'].values)
viewer.add_image(intensity_img, name='intensity_img', colormap='blue', blending='additive')

<Image layer 'intensity_img' at 0x23781b7f1f0>

We can do this for any quantity.

In [26]:
intensity_img = ski.util.map_array(watershedded, results['label'].values, results['area'].values)
viewer.add_image(intensity_img, name='area_img', colormap='blue', blending='additive')

<Image layer 'area_img' at 0x23781c0f670>

# HOMEWORK 2

## Part 1:  Quantify nucleii in "Easy.tif"

Use similar analysis (rolling ball background subtraction, gaussian blur, threshold, label, regionprops_table) to make a label image of the nucleii in "Easy.tif".  

In [60]:
easy_img = ski.io.imread('files/Easy.tif')
backsub_img = backsub_2D(easy_img, radius=10)
blurred_img = ndi.gaussian_filter(backsub_img, 2)
thresholded_img = blurred_img>10
label_img, number_objects = ndi.label(thresholded_img)

viewer.layers.clear()
viewer.add_image(easy_img, name='easy_img', colormap='gray', blending='additive')
viewer.add_image(backsub_img, name='backsub_img', colormap='gray', blending='additive')
viewer.add_image(blurred_img, name='blurred_img', colormap='gray', blending='additive')
viewer.add_image(thresholded_img, name='thresholded_img', colormap='gray', blending='additive')
viewer.add_labels(label_img, name='label_img')

<Labels layer 'label_img' at 0x2379b35a6e0>

Note that a lot of objects are merged that are single objects, we will find ways using watershed to clean this up later, but for now let's filter objects on size. 

Use the ski.measure.regionprops_table() to generate a results table and the ski.util.map_array() function to make "area_img" and then visualize this in napari.

In [66]:
results = pd.DataFrame(ski.measure.regionprops_table(label_img, easy_img, properties=('label', 'area', 'centroid', 'mean_intensity')))
area_img = ski.util.map_array(label_img, results['label'].values, results['area'].values)


## Part 2:  Filter the nucleii by size

### Finding a minimum size

We want to find area limits that let through objects that are single nucleii, but filter out objects that are multiple nucleii and garbage.  

To find the low end (ie the MINIMUM size threshold) we can use our area_img.  Let's just adjust the contrast until the objects that are too small disappear and we are left only with the good ones.

To see the actual VALUE of "contrast" being used (which is actually the object area in our case), we can right click on the "contrast limits" slider.

Set a variable called "min_area" to the value you found.

In [105]:
min_area = 35

### Finding a maximum size

Now we want to find a "max_area" that filters out the double nucleii.

To do this we can play a trick, we can use napari's "gray_r" colormap to visualize the inverse of the area_img.  First we have to play another trick, where we set the area_img's intensity values to be high wherever there is no object (otherwise the background will show as bright).  We will do this using area_img[area_img==0] = np.max(area_img)

area_img==0 returns a binary image, where only the pixels that had value equal to 0 are True and all others are false.  When we ask area_img[..] on a binary image, numpy is smart and returns only the pixels that are True.  We then set all of these pixels to the maximum value in the image.

In [104]:
### THIS CODE STAYS IN THE STUDENT VERSION
area_img[area_img==0] = np.max(area_img)
viewer.add_image(area_img, name='area_img', colormap='gray_r', blending='additive')

<Image layer 'area_img [4]' at 0x237893e73a0>

We want to find a contrast that lets through all the single nucleii, but filters out the double nucleii.  By right clicking on "contrast limits" we can see the actual values of the contrast limits.  Adjusting the upper level is effectively adjusting the area of objects that we are allowing to be display.


Once you find this contrast, use it to filter the results table to only include objects that are smaller than the best max area.

In [70]:
max_area = 175

Alternatively, you can just guess a max_area, and see what results you get below.

### Filtering the objects

Finally, use the "remove_objects" function written above to keep only the objects that are small enough to be single nucleii, then show this new labeled image in napari.

In [72]:
filtered_labels = remove_objects(label_img, min_area, max_area)
viewer.add_labels(filtered_labels, name='filtered_labels')

<Labels layer 'filtered_labels [1]' at 0x2379aedc670>

## Extra credit: Looking at object standard deviations

### Standard deviation of original labels

Recall we can create our own functions for quantifying objects with regionprops, this function finds the standard deviation of the intensity of each object.

In [108]:
### DO NOT REMOVE FROM STUDENT VERSION

def object_stdeviation(mask_img, intensity_img):
    return np.std(intensity_img[mask_img])


Use this function and the "extra_properties" argument of ski.measure.regionprops_table to find the standard deviation of the intensities of each nucleii (in addition to the usual 'label', 'area', 'centroid', 'mean_intensity').  Recall that if you want to quantify intensities, regionprops_table needs both the label image and the intensity image.

In [107]:
results = pd.DataFrame(ski.measure.regionprops_table(filtered_labels, easy_img, properties=('label', 'area', 'centroid', 'mean_intensity'), extra_properties=[object_stdeviation]))

In [109]:
### DO NOT REMOVE FROM STUDENT VERSION
results['SNR'] = results['mean_intensity']/results['object_stdeviation']
results['Type'] = 'Regular'

SNR is a quick way of evaluating how strong our signal to noise is.  Looking at our labels we can see that a lot of pixels that are associated with a nucleus are on the edge where there really is not nucleus.  Including these pixels is going to make our standard deviation much higher that it probably actually is.

### Shrinking the labels

Create a new variable:  shrunk_labels, takes the filtered_labels and shrinks them by 1 pixel.  Add it to napari to make sure it worked.

In [87]:
shrunk_labels = shrink_labels(filtered_labels)

In [88]:
viewer.add_labels(shrunk_labels, name='shrunk_labels')

<Labels layer 'shrunk_labels [1]' at 0x2373d64c370>

Create a new shrunk_results table that is the regionprops_table of the shrunk_labels, including the object_stdeviation as well.

In [89]:
shrunk_results = pd.DataFrame(ski.measure.regionprops_table(shrunk_labels, easy_img, properties=('label', 'area', 'centroid', 'mean_intensity'), extra_properties=[object_stdeviation]))

In [110]:
### DO NOT REMOVE FROM STUDENT VERSION
shrunk_results['SNR'] = shrunk_results['mean_intensity']/shrunk_results['object_stdeviation']
shrunk_results['Type'] = 'Shrunk'

### Plotting results of SNR

If all went well we should get a nice plot of the SNR in shrunk vs unshrunk labels.

In [113]:
### DO NOT REMOVE FROM STUDENT VERSION

stapled_results = pd.concat([results, shrunk_results])
px.box(stapled_results, x='Type', y='SNR', points='all', width=400)


By shrinking our objects so that they only included pixels from the nucleus and not the borders, we drastically increased the signal to noise ratio.